In [6]:
import tools
import yfinance as yf
import pandas as pd

In [2]:
con = tools.connecet_postgres()


In [3]:
t = yf.Ticker("AAPL")
tools.insert_symbol(con, t)    

Eintrag existiert schon


In [4]:
t.quarterly_earnings

,Revenue,Earnings
Quarter,,
3Q2021,83360000000,20551000000
4Q2021,123945000000,34630000000
1Q2022,97278000000,25010000000
2Q2022,82959000000,19442000000


In [5]:
# Tabelle aus Ticker holen
df = t.quarterly_earnings
# Umformen, damit passend zu SQL-Tabellen
symbol = t.info["symbol"]
df["symbol"] = symbol
df["quarter"] = df.index
df["quarter"] = df["quarter"].str[2:] + "Q" + df["quarter"].str[0]
# In Datenbank hochladen
df = df.rename(columns={"Revenue":"revenue","Earnings":"earnings"})
df

df.to_sql("earnings_quarter",con, schema="finance", if_exists="append", index=False)

IntegrityError: (psycopg2.errors.UniqueViolation) FEHLER:  doppelter Schlüsselwert verletzt Unique-Constraint »earnings_quarter_pkey«
DETAIL:  Schlüssel »(symbol, quarter)=(AAPL, 2021Q3)« existiert bereits.

[SQL: INSERT INTO finance.earnings_quarter (revenue, earnings, symbol, quarter) VALUES (%(revenue)s, %(earnings)s, %(symbol)s, %(quarter)s)]
[parameters: ({'revenue': 83360000000, 'earnings': 20551000000, 'symbol': 'AAPL', 'quarter': '2021Q3'}, {'revenue': 123945000000, 'earnings': 34630000000, 'symbol': 'AAPL', 'quarter': '2021Q4'}, {'revenue': 97278000000, 'earnings': 25010000000, 'symbol': 'AAPL', 'quarter': '2022Q1'}, {'revenue': 82959000000, 'earnings': 19442000000, 'symbol': 'AAPL', 'quarter': '2022Q2'})]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

In [ ]:
#df["quarter"].str[2:]
#df["quarter"].str[0]
df["quarter"].str[2:] + "Q" + df["quarter"].str[0]

Quarter
3Q2021    3
4Q2021    4
1Q2022    1
2Q2022    2
Name: quarter, dtype: object

In [7]:
pd.read_sql("select * from finance.earnings_quarter",con)

,symbol,quarter,revenue,earnings
0,AAPL,2021Q3,83360000000,20551000000
1,AAPL,2021Q4,123945000000,34630000000
2,AAPL,2022Q1,97278000000,25010000000
3,AAPL,2022Q2,82959000000,19442000000
